# DISCOGS

Discogs is a database of information about audio recordings, including commercial releases, promotional releases, and bootleg or off-label releases.

Database contents are user-generated, and described in The New York Times as "Wikipedia-like".

While the site was originally created with the goal of becoming the largest online database of electronic music, it now includes releases in all genres and on all formats.



SITE: https://www.discogs.com/

API: https://api.discogs.com/

DOC:

https://python3-discogs-client.readthedocs.io/en/latest/index.html

https://www.discogs.com/developers/

#Installation

In [ ]:
! pip install python3-discogs-client

# Import

In [ ]:
import pandas as pd
import requests
import discogs_client
import operator
import csv
import pprint

#Discogs Client & User token

In [ ]:
d = discogs_client.Client("ExampleApplication/0.1", user_token= "aPYjQukkBxJNCzDiALSJxttKmPMfuLmJDAVOOuKS")

# Mon compte

In [ ]:
me = d.identity()

print(me.id)
print(me.location)
print(me.name)
print(me.url)
#print(me.collection_folders)

2794711
Rennes
TerraPi
https://www.discogs.com/user/Little.Red.Roquet


In [ ]:
print(dir(me))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_known_invalid_keys', 'changes', 'client', 'collection_folders', 'collection_items', 'collection_value', 'data', 'delete', 'fetch', 'home_page', 'id', 'inventory', 'lists', 'location', 'name', 'num_collection', 'num_lists', 'num_wantlist', 'orders', 'previous_request', 'profile', 'rank', 'rating_avg', 'refresh', 'registered', 'releases_contributed', 'save', 'url', 'username', 'wantlist']


In [ ]:
data = []
for item in me.collection_folders[0].releases:
      data.append(item)

data

[<CollectionItemInstance 118668 'Sky Is Clear / Kofea'>,
 <CollectionItemInstance 2469079 'Philharmonics'>,
 <CollectionItemInstance 917960 'Cube'>,
 <CollectionItemInstance 5516756 'Get Down'>,
 <CollectionItemInstance 12631866 'Maxinquaye'>,
 <CollectionItemInstance 14724778 "Tragédie D'Une Trajectoire">,
 <CollectionItemInstance 14883419 'Gangrene'>,
 <CollectionItemInstance 1449729 'Shine On'>,
 <CollectionItemInstance 8383822 'The Dude'>,
 <CollectionItemInstance 1267938 'Any Time Any Place'>,
 <CollectionItemInstance 7685872 'Compton (A Soundtrack By Dr. Dre)'>,
 <CollectionItemInstance 2955328 '2001'>,
 <CollectionItemInstance 5215304 'The Slim Shady LP'>,
 <CollectionItemInstance 5252470 'The Marshall Mathers LP'>,
 <CollectionItemInstance 18669670 'Music To Be Murdered By (Side B)'>,
 <CollectionItemInstance 5199109 'The Marshall Mathers LP 2'>,
 <CollectionItemInstance 403224 'The Eminem Show'>,
 <CollectionItemInstance 1684688 'Head Cleaner'>,
 <CollectionItemInstance 924173

#Import de ma Collection

In [ ]:
# Exporter les données dans un fichier CSV
with open('collection.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([
        'id', 'title', 'artist', 'year',
        'genre', 'style', 'master_id', 'release_country',
        'labels', 'format', 'rating', 'have', 'want', 'url', 'image_url'
    ])

    for release in data:
        # Vérifications pour éviter les erreurs si certaines données sont absentes
        release_data = release.release
        genres = ", ".join(release_data.genres) if release_data.genres else "N/A"
        styles = ", ".join(release_data.styles) if release_data.styles else "N/A"
        master_id = release_data.master.id if release_data.master else "N/A"
        country = release_data.country if release_data.country else "N/A"
        labels = ", ".join([label.name for label in release_data.labels]) if release_data.labels else "N/A"
        formats = ", ".join([", ".join(fmt['descriptions']) for fmt in release_data.formats]) if release_data.formats else "N/A"
        community_rating = (
            release_data.community.rating.average if release_data.community and release_data.community.rating else "N/A"
        )
        community_have = release_data.community.have if release_data.community.have else "N/A"
        community_want = release_data.community.want if release_data.community.want else "N/A"

        image_url = release_data.images[0]["uri"] if release_data.images else "N/A"

        writer.writerow([
            release.id,
            release_data.title,
            release_data.artists[0].name if release_data.artists else "N/A",
            release_data.year,
            genres,
            styles,
            master_id,
            country,
            labels,
            formats,
            community_rating,
            community_have,
            community_want,
            release_data.url,
            image_url
        ])

print("Collection exportée dans 'collection.csv'.")



Collection exportée dans 'collection.csv'.


In [ ]:
df = pd.read_csv("/content/collection.csv")
df.head()

,id,title,artist,year,genre,style,master_id,release_country,labels,format,rating,have,want,url,image_url
0,118668,Sky Is Clear / Kofea,Agoria,2002,Electronic,"Techno, Tech House",369120.0,France,"[PIAS] Recordings, [PIAS] Recordings","12""",4.45,1250,224.0,https://www.discogs.com/release/118668-Agoria-...,https://i.discogs.com/S_XFnlY8MrE5U_1CE1ywljs3...
1,2469079,Philharmonics,Agnes Obel,2010,"Folk, World, & Country",Folk,279920.0,Europe,"[PIAS] Recordings, [PIAS] Recordings","LP, Album",4.40,5271,1052.0,https://www.discogs.com/release/2469079-Agnes-...,https://i.discogs.com/rq0nvSndZ95Rr_6_tTyygLC1...
2,917960,Cube,Le Peuple De L'Herbe,2005,"Electronic, Hip Hop","Drum n Bass, Dub, Breakbeat",133208.0,Belgium,"[PIAS] Recordings, [PIAS] Recordings, Supadope...","LP, Album",4.62,336,181.0,https://www.discogs.com/release/917960-Le-Peup...,https://i.discogs.com/W-IGM9adk0xWEYz3Aq4KDwyo...
3,5516756,Get Down,Gene Chandler,1978,Funk / Soul,"Soul, Disco",95439.0,US,"20th Century Fox Records, Chi Sound Records","LP, Album, Stereo",3.80,118,45.0,https://www.discogs.com/release/5516756-Gene-C...,https://i.discogs.com/_uHS1nVO71KkFkaIYutpTu6p...
4,12631866,Maxinquaye,Tricky,2018,Electronic,"Leftfield, Downtempo, Trip Hop",50297.0,Europe,"4th & Broadway, UMC","LP, Album, Reissue",4.54,2846,635.0,https://www.discogs.com/release/12631866-Trick...,https://i.discogs.com/yCTCNcC4MU7UNolr3OrK1Bgn...


# Export de ma collection

In [ ]:
# load fichier .csv
df = pd.read_csv("/content/collection.csv")

# tri du tableau par nom artiste
df = df.sort_values("artist")

# rest de l'index du tableau
df = df.reset_index()

#suppression de l'ancienne colonne index
df = df.drop("index",axis=1)

#export du nouveau tableau .csv
df.to_csv("my_collection.csv")


# example MC Solaar – Prose Combat
id = 20348713

https://www.discogs.com/release/20348713-MC-Solaar-Prose-Combat

In [ ]:
print(dir(d.release(20348713)))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_known_invalid_keys', 'artists', 'artists_sort', 'changes', 'client', 'community', 'companies', 'country', 'credits', 'data', 'data_quality', 'delete', 'fetch', 'formats', 'genres', 'id', 'images', 'labels', 'marketplace_stats', 'master', 'notes', 'previous_request', 'price_suggestions', 'refresh', 'save', 'status', 'styles', 'thumb', 'title', 'tracklist', 'url', 'videos', 'year']


In [ ]:
print(release.data)

{'id': 20348713, 'resource_url': 'https://api.discogs.com/releases/20348713', 'status': 'Accepted', 'year': 2021, 'uri': 'https://www.discogs.com/release/20348713-MC-Solaar-Prose-Combat', 'artists': [{'name': 'MC Solaar', 'anv': '', 'join': '', 'role': '', 'tracks': '', 'id': 16412, 'resource_url': 'https://api.discogs.com/artists/16412', 'thumbnail_url': 'https://i.discogs.com/F_9BNAyyXR-FMw0jtDt5UG1yv9uYBQndo1TRnwl6vW4/rs:fit/g:sm/q:90/h:600/w:507/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9BLTE2NDEy/LTE2NTM2MzE2MDgt/NzIxNi5qcGVn.jpeg'}], 'artists_sort': 'MC Solaar', 'labels': [{'name': 'Polydor', 'catno': '359903-0', 'entity_type': '1', 'entity_type_name': 'Label', 'id': 1610, 'resource_url': 'https://api.discogs.com/labels/1610', 'thumbnail_url': 'https://i.discogs.com/U0bhkeIQ0b2FTdGabwmzLoNfYYOcXe3Rxd04DdS8c0Y/rs:fit/g:sm/q:90/h:265/w:600/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9MLTM2MzQ2/LTExNzIzMDgzODgu/anBlZw.jpeg'}, {'name': 'Universal Music France', 'catno': '359903-0', 'entity

In [ ]:
release = d.release(20348713)

print(release.id)
print(release.title)
print(release.artists)
print(release.fetch('community'))
print(release.formats)
print(release.genres)
print(release.master)

20348713
Prose Combat
[<Artist 16412 'MC Solaar'>]
{'have': 3333, 'want': 422, 'rating': {'count': 245, 'average': 4.67}, 'submitter': {'username': 'PlexusRecords', 'resource_url': 'https://api.discogs.com/users/PlexusRecords'}, 'contributors': [{'username': 'PlexusRecords', 'resource_url': 'https://api.discogs.com/users/PlexusRecords'}, {'username': 'MadSyro', 'resource_url': 'https://api.discogs.com/users/MadSyro'}, {'username': 'anthonytor', 'resource_url': 'https://api.discogs.com/users/anthonytor'}, {'username': 'boogiedeluxe', 'resource_url': 'https://api.discogs.com/users/boogiedeluxe'}, {'username': 'Arty.gz', 'resource_url': 'https://api.discogs.com/users/Arty.gz'}, {'username': 'Franck-Dijon', 'resource_url': 'https://api.discogs.com/users/Franck-Dijon'}, {'username': 'deenorectah', 'resource_url': 'https://api.discogs.com/users/deenorectah'}, {'username': 'N.a.t.u.r.e', 'resource_url': 'https://api.discogs.com/users/N.a.t.u.r.e'}, {'username': 'djsgribbs', 'resource_url': 'h

In [ ]:

community_details = release.community

print(community_details)
print(community_details.rating) # /5
print(community_details.want)
print(community_details.have)

<CommunityDetails want/have: 422/3332>
<Rating avg: 4.67>
422
3332


In [ ]:
print(d.release(20348713).images[0]["uri"])

https://i.discogs.com/C9aazUEp8wWI7jo85ZK3LAp-y7IOMvOpN0mQuQxyP68/rs:fit/g:sm/q:90/h:600/w:600/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9SLTIwMzQ4/NzEzLTE2MzI1ODU1/NzItOTkyNS5qcGVn.jpeg


#Searching

In [ ]:
results = d.search('Can I borrow a feeling?')
#results = d.search('Can I borrow a feeling?', type='release')
#results = d.search('Can I borrow a feeling?', type='release,master')
#results = d.search('Can I borrow a feeling?', artist='Kirk', type='release')
#results = d.search('Can I borrow a feeling?', genre='Hip Hop')
print(results)

In [ ]:
>>> results = d.search('Stockholm By Night', type='release')
>>> results.pages
1
>>> artist = results[0].artists[0]
>>> artist.name


'The Persuader'

In [ ]:
>>> artist.id
1
>>> artist == d.artist(1)

True

In [ ]:
>>> releases = d.search('Bit Shifter', type='artist')[0].releases[1].\
...     versions[0].labels[0].releases
>>> len(releases)

AttributeError: 'Release' object has no attribute 'versions'

#LOOP

In [ ]:
artist_attribut = ['aliases', 'changes', 'client', 'data', 'data_quality', 'delete', 'fetch', 'groups', 'id', 'images', 'join', 'members', 'name', 'name_variations', 'previous_request', 'profile', 'real_name', 'refresh', 'releases', 'role', 'save', 'url', 'urls']

In [ ]:
release_attribut = ['artists', 'changes', 'client', 'companies', 'country', 'credits', 'data', 'data_quality', 'delete', 'fetch', 'formats', 'genres', 'id', 'images', 'labels', 'marketplace_stats', 'master', 'notes', 'refresh', 'save', 'status', 'styles', 'thumb', 'title', 'tracklist', 'url', 'videos', 'year']

In [ ]:
attributes = artist_attribut
results = []
for i in range(1,2):
  print(d.artist(i))
  print(d.artist(i).releases)
  artist_dict = {}
  for attribute in attributes:
    #print(d.artist(i).attribute)
    artist_dict[attribute] = operator.attrgetter(attribute)(d.artist(i))
  results.append(artist_dict)

<Artist 1 'The Persuader'>


In [ ]:
attributes = ['artists', 'changes', 'client', 'companies', 'country', 'credits', 'data', 'data_quality', 'delete', 'fetch', 'formats', 'genres', 'id', 'images', 'labels', 'marketplace_stats', 'master', 'notes', 'refresh', 'save', 'status', 'styles', 'thumb', 'title', 'tracklist', 'url', 'videos', 'year']
results = []
for i in range(1,2):
  print(d.release(i))
  print(d.release(i).artists)
  artist_dict = {}
  for attribute in attributes:
    #print(d.release(i).attribute)
    artist_dict[attribute] = operator.attrgetter(attribute)(d.release(i))
  results.append(artist_dict)

<Release 1 'Stockholm'>
[<Artist 1 'The Persuader'>]


In [ ]:
results[0]

{'artists': [<Artist 1 'The Persuader'>],
 'changes': {},
 'client': <discogs_client.client.Client at 0x7c1fa7ea66e0>,
 'companies': [<Label 271046 'The Globe Studios'>,
  <Label 56025 'MPO'>,
  <Label 102395 'Tape To Tape'>],
 'country': 'Sweden',
 'credits': [<Artist 507025 'George Cutmaster General'>,
  <Artist 239 'Jesper Dahlbäck'>],
 'data': {'id': 1, 'resource_url': 'https://api.discogs.com/releases/1'},
 'data_quality': 'Needs Vote',
 'delete': <bound method PrimaryAPIObject.delete of <Release 1 'Stockholm'>>,
 'fetch': <bound method PrimaryAPIObject.fetch of <Release 1 'Stockholm'>>,
 'formats': [{'name': 'Vinyl',
   'qty': '2',
   'descriptions': ['12"', '33 ⅓ RPM']}],
 'genres': ['Electronic'],
 'id': 1,
 'images': [{'type': 'primary',
   'uri': 'https://i.discogs.com/Mo72Wf5euwlvzngyek7sRdRV330l8_55Sc-sIZ9ApRI/rs:fit/g:sm/q:90/h:600/w:600/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9SLTEtMTE5/MzgxMjAzMS5qcGVn.jpeg',
   'resource_url': 'https://i.discogs.com/Mo72Wf5euwlvzngyek7s

#test samidou

In [ ]:
print(dir(d.artist(45)))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_known_invalid_keys', 'aliases', 'changes', 'client', 'data', 'data_quality', 'delete', 'fetch', 'groups', 'id', 'images', 'join', 'members', 'name', 'name_variations', 'previous_request', 'profile', 'real_name', 'refresh', 'releases', 'role', 'save', 'url', 'urls']


In [ ]:
phex = d.artist(45)

r = phex.releases

In [ ]:
len(r)

1243

In [ ]:
for element in r :
  print(element)

<Master 870 'Analogue Bubblebath'>
<Master 258478 'Analog Bubblebath Vol 2'>
<Master 565 'Selected Ambient Works 85-92'>
<Master 604 'Digeridoo'>
<Master 919 'Analogue Bubblebath Vol 3'>
<Master 984 'Xylem Tube E.P.'>
<Master 21190 'Time To Find Me'>
<Master 715 'On'>
<Master 32746 'Who Do You Think You Are / Your Head My Voice'>
<Release 15608 'Words & Music'>
<Master 2429 'Analogue Bubblebath 4'>
<Master 481 'Selected Ambient Works Volume II'>
<Release 182628 'In-Store Ambients'>
<Master 20524 'Ventolin E.P'>
<Master 32301 'Raising The Titanic'>
<Master 791 'Donkey Rhubarb'>
<Master 401184 'Melodies From Mars'>
<Master 1725233 'Hangable Auto Bulb EP'>
<Master 845617 'Analogue Bubblebath 5'>
<Master 4984 'Hangable Auto Bulb EP.2'>
<Master 2482 'Classics'>
<Master 461 '...I Care Because You Do'>
<Master 118531 'The Freebase Connection'>
<Master 2459 '51/13 Aphex Singles Collection'>
<Master 2503 '"Girl/Boy" E.P.'>
<Master 510 'Richard D. James Album'>
<Master 12991 'Come To Viddy'>
<Ma

#CHat GPT

In [ ]:
for release in phex.releases:
    # Afficher les informations de chaque sortie
    print(f"Title: {release.title}")
    print(f"Year: {release.year}")
    #print(f"Type: {release.type}")
    print(f"URL: {release.url}")
    print('-' * 30)

Title: Analogue Bubblebath
Year: 1991
URL: https://www.discogs.com/master/870-The-Aphex-Twin-Analogue-Bubblebath
------------------------------
Title: Analog Bubblebath Vol 2
Year: 1991
URL: https://www.discogs.com/master/258478-Aphex-Twin-Analog-Bubblebath-Vol-2
------------------------------
Title: Selected Ambient Works 85-92
Year: 1992
URL: https://www.discogs.com/master/565-Aphex-Twin-Selected-Ambient-Works-85-92
------------------------------
Title: Digeridoo
Year: 1992
URL: https://www.discogs.com/master/604-The-Aphex-Twin-Digeridoo
------------------------------
Title: Analogue Bubblebath Vol 3
Year: 1992
URL: https://www.discogs.com/master/919-AFX-Analogue-Bubblebath-Vol-3
------------------------------
Title: Xylem Tube E.P.
Year: 1992
URL: https://www.discogs.com/master/984-Aphex-Twin-Xylem-Tube-EP
------------------------------
Title: Time To Find Me
Year: 1993
URL: https://www.discogs.com/master/21190-Seefeel-Aphex-Twin-Time-To-Find-Me
------------------------------
Title:

KeyboardInterrupt: 

In [ ]:
z = []
for release in phex.releases:
    # Vérifier si l'objet est de type Release ou Master
    if isinstance(release, discogs_client.models.Master):
        # Si c'est un Master, on peut accéder à ses releases
    z.append(release.title)

KeyboardInterrupt: 

In [ ]:
len(z)

383

In [ ]:
for release in phex.releases:
    if isinstance(release, discogs_client.Master):
        # Vérifiez les formats et descriptions
        formats = release.data.get("formats", [])
        descriptions = [desc for f in formats for desc in f.get("descriptions", [])]

        # Filtre : inclure uniquement les albums
        if "Album" in descriptions and not any(desc in ["Compilation", "Single", "EP"] for desc in descriptions):
            albums.append({
                'title': release.title,
                'year': release.year,
                'id': release.id
            })

# Tri des albums par année de sortie
albums_sorted = sorted(albums, key=lambda x: x['year'])

# Affichage des résultats
for album in albums_sorted:
    print(f"{album['year']} - {album['title']} (ID: {album['id']})")

In [ ]:
len(albums)

383